In [25]:
%%time
from functools import partial
from collections import defaultdict, namedtuple
import numpy as np
import pandas as pd
import scipy
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from itertools import product
import time

from video699.screen.semantic_segmentation.fastai_detector import *
from video699.screen.semantic_segmentation.common import *
from video699.screen.semantic_segmentation.postprocessing import *
from video699.screen.semantic_segmentation.evaluation import *

Split = namedtuple('Split', 'train valid')
all_lectures = [video.filename for video in ALL_VIDEOS]

CPU times: user 235 µs, sys: 47 µs, total: 282 µs
Wall time: 290 µs


In [64]:
def get_best_params(df):
    unhashable_columns = ['frozen_lr', 'unfrozen_lr', 'base_factors', 'erode_dilate_factors']
    df[unhashable_columns] = df[unhashable_columns].astype(str)
    df['wrong_count'] = df['wrong_count'].astype(int)
    best_params = df.groupby(train_params + method_params).mean().sort_values(by=['wrong_count', 'iou']).iloc[0].name

    converted_params = []
    for i, par in enumerate(best_params):
        if isinstance(par, np.int64) or isinstance(par, np.float64):
            converted_params.append(par.item())
        else:
            converted_params.append(par)
    best_params = tuple(converted_params)

    best_methods = dict(zip(method_params, best_params[-len(method_params):]))
    best_train_params_dict = dict(zip(train_params, best_params[:len(train_params)]))
    best_train_params_dict['frozen_lr'] = eval(best_train_params_dict['frozen_lr'])
    best_train_params_dict['unfrozen_lr'] = eval(best_train_params_dict['unfrozen_lr'])
    best_methods['base_factors'] = eval(best_methods['base_factors'])
    best_methods['erode_dilate_factors'] = eval(best_methods['erode_dilate_factors'])
    return best_methods, best_train_params_dict

In [65]:
kf = KFold(n_splits=5, shuffle=True, random_state=123)
outer_splits = {}
for i, split in tqdm(enumerate(kf.split(all_lectures))):
    other_lectures = [all_lectures[index] for index in split[0]]
    test_lectures = [all_lectures[index] for index in split[1]]
    outer_splits[i] = Split(train=other_lectures, valid=test_lectures)

5it [00:00, 5669.51it/s]


## Cross validation summary

In [66]:
df_cv = pd.read_csv("cross_validation_results.csv", usecols = [i for i in range(1, 21)])

In [67]:
get_best_params(df_cv)

({'base': True,
  'erode_dilate': True,
  'ratio_split': True,
  'base_lower_bound': 7,
  'base_upper_bound': 70,
  'base_factors': [0.1, 0.01],
  'erode_dilate_lower_bound': 7,
  'erode_dilate_upper_bound': 70,
  'erode_dilate_factors': [0.1, 0.01],
  'erode_dilate_iterations': 100,
  'ratio_split_lower_bound': 0.9,
  'ratio_split_upper_bound': 1.5},
 {'batch_size': 8,
  'resize_factor': 2,
  'frozen_epochs': 9,
  'unfrozen_epochs': 10,
  'frozen_lr': 0.0001,
  'unfrozen_lr': slice(0.0001, 0.0002, None)})

In [47]:
percentage_of_missed_screens  = df_cv['wrong_count'].mean() / len(all_frames) * 100
percentage_of_missed_screens

2.93398533007335

In [48]:
iou_score = df_cv['iou'].mean() * 100
iou_score

68.42506772524365

### Outer splits with most missed screens

In [49]:
for i in list(df_cv[df_cv['wrong_count'] > 5].index):
    print(outer_splits[i].valid)

['PB001-D1-20121016.avi', 'MB103-D1-20161122.mp4', 'IB111-D1-20151211.mp4', 'MA010-D1-20151015.mp4']
['PV112-D2-20130311.avi', 'PB156-D1-20130312.avi', 'PB069-D2-20140305.mp4']


### Outer splits with worst iou

In [50]:
for i in list(df_cv[df_cv['iou'] < 0.6].index):
    print(outer_splits[i].valid)

['PV112-D2-20130311.avi', 'PB156-D1-20130312.avi', 'PB069-D2-20140305.mp4']


## Model selection summary

In [51]:
df_ms = pd.read_csv("model_selection.csv", usecols = [i for i in range(1, 21)])

In [56]:
unhashable_columns = ['frozen_lr', 'unfrozen_lr', 'base_factors', 'erode_dilate_factors']
df_ms[unhashable_columns] = df_ms[unhashable_columns].astype(str)
df_ms['wrong_count'] = df_ms['wrong_count'].astype(int)
best_params = df_ms.groupby(train_params + method_params).mean().sort_values(by=['wrong_count', 'iou']).iloc[0].name

converted_params = []
for i, par in enumerate(best_params):
    if isinstance(par, np.int64) or isinstance(par, np.float64):
        converted_params.append(par.item())
    else:
        converted_params.append(par)
best_params = tuple(converted_params)

best_methods = dict(zip(method_params, best_params[-len(method_params):]))
best_train_params_dict = dict(zip(train_params, best_params[:len(train_params)]))
best_train_params_dict['frozen_lr'] = eval(best_train_params_dict['frozen_lr'])
best_train_params_dict['unfrozen_lr'] = eval(best_train_params_dict['unfrozen_lr'])
best_methods['base_factors'] = eval(best_methods['base_factors'])
best_methods['erode_dilate_factors'] = eval(best_methods['erode_dilate_factors'])

In [58]:
best_methods, best_train_params_dict

({'base': True,
  'erode_dilate': True,
  'ratio_split': True,
  'base_lower_bound': 7,
  'base_upper_bound': 70,
  'base_factors': [0.1, 0.01],
  'erode_dilate_lower_bound': 7,
  'erode_dilate_upper_bound': 70,
  'erode_dilate_factors': [0.1, 0.01],
  'erode_dilate_iterations': 40,
  'ratio_split_lower_bound': 0.7,
  'ratio_split_upper_bound': 1.5},
 {'batch_size': 8,
  'resize_factor': 2,
  'frozen_epochs': 2,
  'unfrozen_epochs': 3,
  'frozen_lr': 0.0001,
  'unfrozen_lr': slice(0.0001, 0.0002, None)})